# Day 24
## Part 1


In [8]:
from collections import namedtuple
from pyrsistent import pmap

Gate = namedtuple("Gate", "input1 op input2")

ops = {
    "AND": lambda x, y: x & y,
    "OR": lambda x, y: x | y,
    "XOR": lambda x, y: x ^ y
}

def parse_data(s):
    chunks = s.strip().split("\n\n")

    values = {}
    for line in chunks[0].splitlines():
        wire, value = line.split(": ")
        values[wire] = int(value)

    gates = {}
    for line in chunks[1].splitlines():
        inputs, output = line.split(" -> ")
        gates[output] = Gate(*inputs.split())

    return pmap(values), pmap(gates)

test_data = parse_data("""x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02""")

test_data

(pmap({'y02': 0, 'x01': 1, 'y00': 0, 'x00': 1, 'y01': 1, 'x02': 1}),
 pmap({'z01': Gate(input1='x01', op='XOR', input2='y01'), 'z02': Gate(input1='x02', op='OR', input2='y02'), 'z00': Gate(input1='x00', op='AND', input2='y00')}))

In [23]:
import graphlib

def part_1(data):
    values, gates = data

    ts = graphlib.TopologicalSorter()
    for g in gates:
        ts.add(g, gates[g].input1, gates[g].input2)

    for wire in ts.static_order():
        if wire not in values:
            g = gates[wire]
            values = values.set(
                wire, ops[g.op](values[g.input1], values[g.input2])
            )

    zs = [wire for wire in values if wire.startswith("z")]
    binary = "".join(str(values[x]) for x in reversed(sorted(zs)))
    return int(binary, 2)
    
part_1(test_data)

4

In [24]:
test_data_2 = parse_data("""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj""")

part_1(test_data_2)

2024

In [25]:
data = parse_data(open("input").read())

part_1(data)

69201640933606

## Part 2

Chunky! How many bits are in the values to be added?

In [29]:
values, gates = data
len([v for v in values if v.startswith("x")])

45